<a href="https://colab.research.google.com/github/jaadu-1/Algo-trading/blob/main/Product_!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

krish39696_product_track_path = kagglehub.dataset_download('krish39696/product-track')

print('Data source import complete.')


In [ ]:
import pandas as pd

# Load offer data
offer_df = pd.read_parquet('/kaggle/input/product-track/amex_offers_data.parquet')  # Adjust path as needed

# Preview
offer_df.head()
offer_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2004812 entries, 0 to 2004811
Data columns (total 55 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   customer_id   object
 1   offer_id      object
 2   event_ts      object
 3   event_dt      object
 4   offer_action  object
 5   var_1         object
 6   var_2         object
 7   var_3         object
 8   var_4         object
 9   var_5         object
 10  var_6         object
 11  var_7         object
 12  var_8         object
 13  var_9         object
 14  var_10        object
 15  var_11        object
 16  var_12        object
 17  var_13        object
 18  var_14        object
 19  var_15        object
 20  var_16        object
 21  var_17        object
 22  var_18        object
 23  var_19        object
 24  var_20        object
 25  var_21        object
 26  var_22        object
 27  var_23        object
 28  var_24        object
 29  var_25        object
 30  var_26        object
 31  var_27        object
 32

In [ ]:
offer_df.columns


Index(['customer_id', 'offer_id', 'event_ts', 'event_dt', 'offer_action',
       'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7', 'var_8',
       'var_9', 'var_10', 'var_11', 'var_12', 'var_13', 'var_14', 'var_15',
       'var_16', 'var_17', 'var_18', 'var_19', 'var_20', 'var_21', 'var_22',
       'var_23', 'var_24', 'var_25', 'var_26', 'var_27', 'var_28', 'var_29',
       'var_30', 'var_31', 'var_32', 'var_33', 'var_34', 'var_35', 'var_36',
       'var_37', 'var_38', 'var_39', 'var_40', 'var_41', 'var_42', 'var_43',
       'var_44', 'var_45', 'var_46', 'var_47', 'var_48', 'var_49', 'var_50'],
      dtype='object')

In [ ]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_parquet('/kaggle/input/product-track/amex_offers_data.parquet')
submission = pd.read_csv('/kaggle/input/product-track/6855142f07670_submission_template.csv')  # still used, not saved

# -------------------------------
# Step 1: Clean Dates
# -------------------------------
df['event_dt'] = pd.to_datetime(df['event_dt'], errors='coerce', infer_datetime_format=True)



# -------------------------------
# Step 2: Convert important variables to numeric
# -------------------------------
vars_to_convert = [
    'var_9',  # airline interest
    'var_20', 'var_27', 'var_50',  # airline/travel spend indicators
    'var_31',  # clicked
    'var_32',  # min spend for DOE
    'var_34',  # discount value
    'var_36',  # reward points per dollar
    'var_37', 'var_38', 'var_39', 'var_40', 'var_41', 'var_42', 'var_43'  # CTR-related
]

for col in vars_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# -------------------------------
# Step 3: Flags and Derived Columns
# -------------------------------
df['is_clicked'] = df['var_31'].fillna(0).astype(int)
df['discount_value'] = df['var_34']

# Airline-related offer flag
df['is_airline_offer'] = (
    (df['var_9'] > 0.5) |
    (df['var_20'] > 0) |
    (df['var_27'] > 0) |
    (df['var_50'] == 1)
).astype(int)

# -------------------------------
# Step 4: Define Active Users (30+ days span)
# -------------------------------
user_activity = df.groupby('customer_id')['event_dt'].agg(['min', 'max']).reset_index()
user_activity['active_days'] = (user_activity['max'] - user_activity['min']).dt.days
user_activity['is_active'] = user_activity['active_days'] >= 5

# Merge activity flag
df = df.merge(user_activity[['customer_id', 'is_active']], on='customer_id', how='left')

# -------------------------------
# Step 5: Q1.1 - Are active users more likely to choose airline offers?
# -------------------------------
active_df = df[df['is_active'] == True]
airline_ctr = active_df[active_df['is_airline_offer'] == 1]['is_clicked'].mean()
non_airline_ctr = active_df[active_df['is_airline_offer'] == 0]['is_clicked'].mean()
q1_1 = 'TRUE' if airline_ctr > non_airline_ctr else 'FALSE'

# -------------------------------
# Step 6: Q1.2 - Do active users save more from discounts?
# -------------------------------
active_discount = active_df[active_df['is_clicked'] == 1]['discount_value'].mean()
inactive_discount = df[(df['is_active'] == False) & (df['is_clicked'] == 1)]['discount_value'].mean()
q1_2 = 'TRUE' if active_discount > inactive_discount else 'FALSE'

# -------------------------------
# Step 7: Q1.3 - Unique active users who clicked on airline offers
# -------------------------------
q1_3 = df[(df['is_active']) & (df['is_clicked'] == 1) & (df['is_airline_offer'] == 1)]['customer_id'].nunique()

# -------------------------------
# Step 8: Print Results
# -------------------------------
print(f"Q1.1 ➤ Active users more likely to choose airline offers? → {q1_1}")
print(f"Q1.2 ➤ Active users save more from discounts? → {q1_2}")
print(f"Q1.3 ➤ Unique active users who clicked airline offers → {q1_3}")


/tmp/ipykernel_35/4176316250.py:11: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['event_dt'] = pd.to_datetime(df['event_dt'], errors='coerce', infer_datetime_format=True)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)


Q1.1 ➤ Active users more likely to choose airline offers? → TRUE
Q1.2 ➤ Active users save more from discounts? → TRUE
Q1.3 ➤ Unique active users who clicked airline offers → 2688


In [ ]:
for col in ['var_48', 'var_49', 'var_50']:
    df_q2[col] = pd.to_numeric(df_q2[col], errors='coerce').fillna(0).astype(int)


In [ ]:
print("Services offers:", df_q2['var_48'].sum())
print("Shopping offers:", df_q2['var_49'].sum())
print("Travel offers:", df_q2['var_50'].sum())


Services offers: 103709
Shopping offers: 785022
Travel offers: 76924


In [ ]:
travel_df = df_q2[df_q2['var_50'] == 1]

high_group = travel_df[travel_df['var_37'] >= df_q2['var_37'].median()]
low_group = travel_df[travel_df['var_37'] < df_q2['var_37'].median()]

print("Travel High Group CTR:", high_group['var_31'].mean())
print("Travel Low Group CTR:", low_group['var_31'].mean())


Travel High Group CTR: 1.0
Travel Low Group CTR: 1.0


In [ ]:
print("Travel total:", len(travel_df))
print("High group count:", len(high_group))
print("Low group count:", len(low_group))


Travel total: 76924
High group count: 75021
Low group count: 1903


In [ ]:
print("Unique click values in travel group:", travel_df['var_31'].unique())


Unique click values in travel group: [ 1. nan]


In [ ]:
import numpy as np
import pandas as pd

# Ensure columns are numeric and clean
df['var_31'] = pd.to_numeric(df['var_31'], errors='coerce').fillna(0).astype(int)
df['var_37'] = pd.to_numeric(df['var_37'], errors='coerce')

for col in ['var_48', 'var_49', 'var_50']:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

# Filter data for non-zero valid discount value per dollar and valid click values
df_q2 = df[
    df['var_37'].notna() &
    (df['var_37'] > 0) &
    (df['var_31'].isin([0, 1]))
].copy()

# Calculate median of discount per dollar (50th percentile)
discount_median = df_q2['var_37'].median()

# Label offers into high/low discount-per-dollar buckets
df_q2['disc_value_group'] = np.where(df_q2['var_37'] >= discount_median, 'high', 'low')

# Set up category mapping
category_map = {
    'var_48': 'Services',
    'var_49': 'Shopping',
    'var_50': 'Travel'
}

q2_results = {}

# Loop over each offer category
for var, category_name in category_map.items():
    cat_df = df_q2[df_q2[var] == 1]

    if cat_df.empty:
        q2_results[category_name] = 0.000
        continue

    high_ctr = cat_df[cat_df['disc_value_group'] == 'high']['var_31'].mean()
    low_ctr = cat_df[cat_df['disc_value_group'] == 'low']['var_31'].mean()

    ctr_diff = np.floor((high_ctr - low_ctr) * 1000) / 1000  # floor to 3 decimals
    q2_results[category_name] = ctr_diff

# Print final results
print("Q2 ➤ CTR Difference (High Discount - Low Discount):")
for category, diff in q2_results.items():
    print(f"{category}: {diff}")


Q2 ➤ CTR Difference (High Discount - Low Discount):
Services: 0.002
Shopping: -0.024
Travel: -0.008


In [ ]:
# Convert all relevant columns to numeric
spend_cols = ['var_17', 'var_18', 'var_19', 'var_31', 'var_36']
for col in spend_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Step 1: Calculate Max Spend per Customer
df['max_spend'] = df[['var_17', 'var_18', 'var_19']].max(axis=1)

# Step 2: Get Customer-Level Spend (take max_spend per customer)
customer_spend = df.groupby('customer_id')['max_spend'].max().dropna()

# Step 3: Split into 3 equal segments
quantiles = customer_spend.quantile([1/3, 2/3])
low_thres = quantiles[1/3]
high_thres = quantiles[2/3]

def label_segment(spend):
    if spend <= low_thres:
        return 'LOW'
    elif spend <= high_thres:
        return 'MEDIUM'
    else:
        return 'HIGH'

customer_segments = customer_spend.apply(label_segment).reset_index()
customer_segments.columns = ['customer_id', 'segment']

# Merge segment info back to original df
df_q3 = df.merge(customer_segments, on='customer_id', how='inner')

# Step 4.1: Avg reward point/dollar (var_36) for offers shown
avg_reward_offered = df_q3.groupby('segment')['var_36'].mean().round(3)

# Step 4.2: Avg reward point/dollar for offers clicked
clicked = df_q3[df_q3['var_31'] == 1]
avg_reward_clicked = clicked.groupby('segment')['var_36'].mean().round(3)

# Step 4.3: Participation Rate = clicks / shown
click_rate = df_q3.groupby('segment')['var_31'].mean().round(3)

# Step 5: Find best performing segment
best_segment = click_rate.idxmax()

# Output everything
print("Q3.1 ➤ Avg Reward Point per Dollar (Offers Shown):")
print(avg_reward_offered)
print("\nQ3.2 ➤ Avg Reward Point per Dollar (Offers Clicked):")
print(avg_reward_clicked)
print("\nQ3.3 ➤ Best Performing Segment (based on click rate):", best_segment)


Q3.1 ➤ Avg Reward Point per Dollar (Offers Shown):
segment
HIGH      12.789
LOW       12.786
MEDIUM    12.838
Name: var_36, dtype: float64

Q3.2 ➤ Avg Reward Point per Dollar (Offers Clicked):
segment
HIGH      12.084
LOW       10.375
MEDIUM    11.389
Name: var_36, dtype: float64

Q3.3 ➤ Best Performing Segment (based on click rate): LOW


In [ ]:
from datetime import datetime, timedelta

# 1. Filter data up to 7th Nov 2023
cutoff_date = pd.to_datetime("2023-11-07")
df['event_dt'] = pd.to_datetime(df['event_dt'], errors='coerce')
df_q4 = df[df['event_dt'] <= cutoff_date].copy()

# 2. Calculate CTR per offer_id
offer_stats = df_q4.groupby('offer_id').agg(
    views=('var_31', 'count'),
    clicks=('var_31', 'sum')
)
offer_stats['ctr'] = offer_stats['clicks'] / offer_stats['views']

# 3. Only consider offers with above-average views
avg_views = offer_stats['views'].mean()
top_offers = offer_stats[offer_stats['views'] > avg_views].sort_values('ctr', ascending=False).head(10)
top_offer_ids = top_offers.index.tolist()

print("Top 10 Offer IDs:", top_offer_ids)

# 4. Map Offer ID to Offer Category (pick first category column with 1)
category_cols = ['var_44', 'var_45', 'var_46', 'var_47', 'var_48', 'var_49', 'var_50']
category_names = {
    'var_44': 'Business',
    'var_45': 'Dining',
    'var_46': 'Entertainment',
    'var_47': 'Retail',
    'var_48': 'Services',
    'var_49': 'Shopping',
    'var_50': 'Travel'
}

top_offers_df = df_q4[df_q4['offer_id'].isin(top_offer_ids)].copy()

def get_offer_category(row):
    for var in category_cols:
        if pd.to_numeric(row.get(var, 0), errors='coerce') == 1:
            return category_names[var]
    return "Unknown"

top_offers_df['offer_category'] = top_offers_df.apply(get_offer_category, axis=1)
offer_categories = top_offers_df.groupby('offer_id')['offer_category'].first()

# 5. Average 3-month spend of users who clicked these offers
top_clicks = top_offers_df[top_offers_df['var_31'] == 1].copy()
top_clicks['3m_spend'] = top_clicks[['var_17', 'var_18', 'var_19']].max(axis=1)
avg_spend = top_clicks.groupby('offer_id')['3m_spend'].mean().mean()
avg_spend = np.floor(avg_spend * 1000) / 1000

# 6. Calculate CTR in last 30 days before 7th Nov 2023 (i.e., since 8th Oct 2023)
recent_start = cutoff_date - timedelta(days=30)
recent_df = df[
    df['offer_id'].isin(top_offer_ids) &
    (df['event_dt'] >= recent_start) &
    (df['event_dt'] <= cutoff_date)
].copy()

recent_ctr = recent_df.groupby('offer_id')['var_31'].mean().round(3)

# Final output
print("\nQ4.1 ➤ Top 10 Offer ID : Offer Category Mapping")
for oid in top_offer_ids:
    print(f"{oid}: {offer_categories.get(oid, 'Unknown')}")

print(f"\nQ4.2 ➤ Average 3-Month Spend for these offers: {avg_spend}")

print("\nQ4.3 ➤ Offer ID : 30-day CTR Mapping")
for oid in top_offer_ids:
    print(f"{oid}: {recent_ctr.get(oid, 0.000)}")


Top 10 Offer IDs: ['760457', '498907', '313795', '72053', '82025', '23311', '7337', '26175', '331980', '26379']

Q4.1 ➤ Top 10 Offer ID : Offer Category Mapping
760457: Unknown
498907: Travel
313795: Travel
72053: Shopping
82025: Travel
23311: Shopping
7337: Shopping
26175: Dining
331980: Shopping
26379: Shopping

Q4.2 ➤ Average 3-Month Spend for these offers: 3853.782

Q4.3 ➤ Offer ID : 30-day CTR Mapping
760457: 0.168
498907: 0.153
313795: 0.145
72053: 0.14
82025: 0.138
23311: 0.126
7337: 0.117
26175: 0.115
331980: 0.111
26379: 0.11


In [ ]:
# Step 1: Filter records as of 14th Nov 2023 (last 30 days window)
cutoff_q5 = pd.to_datetime("2023-11-14")
df['event_dt'] = pd.to_datetime(df['event_dt'], errors='coerce')
df_q5 = df[
    (df['event_dt'] <= cutoff_q5) &
    (df['event_dt'] >= cutoff_q5 - pd.Timedelta(days=30)) &
    (pd.to_numeric(df['var_14'], errors='coerce').fillna(0) > 0)  # participated in email
].copy()

# Step 2: Type casting
df_q5['var_31'] = pd.to_numeric(df_q5['var_31'], errors='coerce').fillna(0).astype(int)
df_q5['var_14'] = pd.to_numeric(df_q5['var_14'], errors='coerce')
df_q5['var_15'] = pd.to_numeric(df_q5['var_15'], errors='coerce')
df_q5['var_13'] = pd.to_numeric(df_q5['var_13'], errors='coerce')

# Step 3: Customer-level stats
cust_stats = df_q5.groupby('customer_id').agg(
    total_clicks=('var_31', 'sum'),
    total_views=('var_31', 'count'),
    channels=('var_13', 'max'),
    email_sent=('var_14', 'max'),
    email_clicks=('var_15', 'max')
)

cust_stats['offer_ctr'] = cust_stats['total_clicks'] / cust_stats['total_views']
top15_customers = cust_stats.sort_values('offer_ctr', ascending=False).head(15)

# Step 4: For each of top 15, get:
# - Clicks per channel
# - Email CTR
# - Top offer category
top15_ids = top15_customers.index.tolist()
df_top15 = df_q5[df_q5['customer_id'].isin(top15_ids)].copy()

# Category mapping
category_cols = ['var_44', 'var_45', 'var_46', 'var_47', 'var_48', 'var_49', 'var_50']
category_names = {
    'var_44': 'Business',
    'var_45': 'Dining',
    'var_46': 'Entertainment',
    'var_47': 'Retail',
    'var_48': 'Services',
    'var_49': 'Shopping',
    'var_50': 'Travel'
}

# Determine top-performing category per customer based on offer clicks
df_top15['clicked'] = df_top15['var_31'] == 1

def get_top_category(group):
    cat_clicks = {}
    for col in category_cols:
        cat_clicks[col] = group[group[col] == 1]['clicked'].sum()
    top_col = max(cat_clicks, key=cat_clicks.get)
    return category_names[top_col]

top_categories = df_top15.groupby('customer_id').apply(get_top_category)

# Final outputs:
print("Q5.1 ➤ CustomerID : Offer Clicks per Channel Ratio")
for cid in top15_ids:
    clicks = top15_customers.loc[cid, 'total_clicks']
    channels = top15_customers.loc[cid, 'channels']
    ratio = np.floor((clicks / channels) * 1000) / 1000 if channels > 0 else 0.0
    print(f"{cid}: {ratio}")

print("\nQ5.2 ➤ CustomerID : Top Performing Offer Category")
for cid in top15_ids:
    print(f"{cid}: {top_categories[cid]}")

print("\nQ5.3 ➤ CustomerID : Email CTR (Email Clicks / Sent)")
for cid in top15_ids:
    sent = top15_customers.loc[cid, 'email_sent']
    clicked = top15_customers.loc[cid, 'email_clicks']
    ratio = np.floor((clicked / sent) * 1000) / 1000 if sent > 0 else 0.0
    print(f"{cid}: {ratio}")


Q5.1 ➤ CustomerID : Offer Clicks per Channel Ratio
1686517: 16.23
1674697: 0.45
1521739: 0.126
1407212: 0.103
1672300: 2.194
1006105: 0.68
1391327: 0.0
1077553: 2.075
1683309: 2.695
1846042: 1.288
1500394: 123.0
1313065: 6.0
1282059: 0.787
1598859: 2.75
1207550: 15.666

Q5.2 ➤ CustomerID : Top Performing Offer Category
1686517: Shopping
1674697: Shopping
1521739: Travel
1407212: Shopping
1672300: Shopping
1006105: Shopping
1391327: Shopping
1077553: Shopping
1683309: Shopping
1846042: Shopping
1500394: Shopping
1313065: Shopping
1282059: Shopping
1598859: Shopping
1207550: Shopping

Q5.3 ➤ CustomerID : Email CTR (Email Clicks / Sent)
1686517: nan
1674697: nan
1521739: nan
1407212: 1.6
1672300: nan
1006105: nan
1391327: nan
1077553: nan
1683309: nan
1846042: nan
1500394: nan
1313065: nan
1282059: nan
1598859: 1.25
1207550: nan


/tmp/ipykernel_35/959145475.py:57: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_categories = df_top15.groupby('customer_id').apply(get_top_category)


In [ ]:
# Ensure datetime is correct
df['event_dt'] = pd.to_datetime(df['event_dt'], errors='coerce')
df['var_14'] = pd.to_numeric(df['var_14'], errors='coerce').fillna(0)
df['var_15'] = pd.to_numeric(df['var_15'], errors='coerce').fillna(0)
df['var_31'] = pd.to_numeric(df['var_31'], errors='coerce').fillna(0)

# Sort data per user over time
df_q6 = df.sort_values(['customer_id', 'event_dt'])

# Step 1 ➤ Compute EWMA per customer per date
def compute_ewma_click_prob(group):
    return group['var_31'].ewm(alpha=0.5).mean()

df_q6['ewma_click_prob'] = df_q6.groupby('customer_id').apply(compute_ewma_click_prob).reset_index(level=0, drop=True)

# Step 2 ➤ P(Click | Email Sent)
# We filter only rows where email was sent
email_sent_df = df_q6[df_q6['var_14'] > 0]
p_click_given_email = np.floor(email_sent_df['ewma_click_prob'].mean() * 1000) / 1000

# Step 3 ➤ P(Click | Clicked in Past)
# We filter rows where past click count > 0 (rolling clicks)
past_click_df = df_q6.groupby('customer_id')['var_31'].cumsum()
clicked_before_df = df_q6[past_click_df > 0]
p_click_given_past = np.floor(clicked_before_df['ewma_click_prob'].mean() * 1000) / 1000

# Print results
print("Q6.1 ➤ P(Click | Email Offer Sent):", p_click_given_email)
print("Q6.2 ➤ P(Click | Clicked in Past):", p_click_given_past)


/tmp/ipykernel_35/3846757762.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_q6['ewma_click_prob'] = df_q6.groupby('customer_id').apply(compute_ewma_click_prob).reset_index(level=0, drop=True)


Q6.1 ➤ P(Click | Email Offer Sent): 0.047
Q6.2 ➤ P(Click | Clicked in Past): 0.095


In [ ]:
import pandas as pd

# Load submission template
template_path = "/kaggle/input/product-track/6855142f07670_submission_template.csv"
submission = pd.read_csv(template_path, encoding="ISO-8859-1")

# Clean up question and part IDs
submission["question_id"] = submission["question_id"].astype(str).str.extract(r'(Q\d+)')
submission["part_id"] = submission["part_id"].astype(str).str.extract(r'(\d+)')

submission.reset_index(drop=True, inplace=True)

# === ✅ ANSWER DICTIONARY ===
answers = {
    ("Q1", "1"): "TRUE",
    ("Q1", "2"): "TRUE",
    ("Q1", "3"): 2688,

    ("Q2", "1"): 0.002,
    ("Q2", "2"): -0.024,
    ("Q2", "3"): -0.008,

    ("Q3", "1"): [12.786, 12.838, 12.789],  # Offers shown
    ("Q3", "2"): [10.375, 11.389, 12.084],  # Offers clicked
    ("Q3", "3"): "LOW",

    ("Q4", "1"): [
        "760457:Unknown", "498907:Travel", "313795:Travel", "72053:Shopping",
        "82025:Travel", "23311:Shopping", "7337:Shopping", "26175:Dining",
        "331980:Shopping", "26379:Shopping"
    ],
    ("Q4", "2"): 3853.782,
    ("Q4", "3"): [
        "760457:0.168", "498907:0.153", "313795:0.145", "72053:0.140",
        "82025:0.138", "23311:0.126", "7337:0.117", "26175:0.115",
        "331980:0.111", "26379:0.110"
    ],

    ("Q5", "1"): [
        "1686517:16.23", "1674697:0.45", "1521739:0.126", "1407212:0.103",
        "1672300:2.194", "1006105:0.68", "1391327:0.0", "1077553:2.075",
        "1683309:2.695", "1846042:1.288", "1500394:123.0", "1313065:6.0",
        "1282059:0.787", "1598859:2.75", "1207550:15.666"
    ],
    ("Q5", "2"): [
        "1686517:Shopping", "1674697:Shopping", "1521739:Travel", "1407212:Shopping",
        "1672300:Shopping", "1006105:Shopping", "1391327:Shopping", "1077553:Shopping",
        "1683309:Shopping", "1846042:Shopping", "1500394:Shopping", "1313065:Shopping",
        "1282059:Shopping", "1598859:Shopping", "1207550:Shopping"
    ],
    ("Q5", "3"): [
        "1686517:nan", "1674697:nan", "1521739:nan", "1407212:1.6", "1672300:nan",
        "1006105:nan", "1391327:nan", "1077553:nan", "1683309:nan", "1846042:nan",
        "1500394:nan", "1313065:nan", "1282059:nan", "1598859:1.25", "1207550:nan"
    ],

    ("Q6", "1"): 0.047,
    ("Q6", "2"): 0.095,
}

# === ✅ Fill in the submission file ===
filled = 0
row_counters = {}

for idx, row in submission.iterrows():
    q = row["question_id"]
    subq = row["part_id"]
    key = (q, subq)

    if key in answers:
        val = answers[key]
        if isinstance(val, list):
            if key not in row_counters:
                row_counters[key] = 0
            if row_counters[key] < len(val):
                submission.at[idx, "Answer"] = val[row_counters[key]]
                row_counters[key] += 1
                filled += 1
        else:
            submission.at[idx, "Answer"] = val
            filled += 1

print(f"✅ Successfully filled {filled} answers.")

# === ✅ Save to output ===
output_path = "/kaggle/working/final_submission.csv"
submission.to_csv(output_path, index=False)
print(f"📁 Submission file saved at: {output_path}")


✅ Successfully filled 81 answers.
📁 Submission file saved at: /kaggle/working/final_submission.csv
